<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/novel_data/50%E6%9C%AC%E7%BB%8F%E5%85%B8%E5%B0%8F%E8%AF%B4%E6%95%B0%E6%8D%AE%E6%9E%84%E9%80%A0%EF%BC%88%E5%90%8E%E5%A4%84%E7%90%86).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 尝试读取文件，看第一行是什么小说
- [x] 决定一本自己读过的小说
- [x] 使用replace_recalled_dialogue生成新的小说
- [x] 进行拼接
- [x] 使用extract_speaker进行对话人确定
- [x] 合并连续相同的speaker
- [x] 随机读取一些数据，看看qwen和chatgpt3.5的token ratio 用后者会快一些
- [x] 确定每个语料最大的合适长度 感觉卡qwen 600 长度比较合适
- [x] 在刚才那个小说下面，选取一个人，从后向前切割rag语料 人物记忆库
- [ ] 确定哪些长度的话适合作为训练数据
- [x] 用贪心法，决定每个output所对应的rag，每个数据尽可能参与平均次数input
- [ ] 为了降低实验难度，0.4直接使用默认prompt，即 你扮演 xx 中的 xx
- [ ] 拓展一本小说的所有人物
- [ ] 检查一本小说的数据
- [ ] 拓展到50本小说

In [ ]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/CardBuild/exp0212

novel_50_extract.jsonl	source2new_book.pkl


In [ ]:
def parse_id(id_str):
    pos = id_str.rfind("_")
    return id_str[:pos], int( id_str[pos+1:] )

# print( parse_id(data["id"]))

In [ ]:
input_file = "/content/drive/MyDrive/CardBuild/exp0212/novel_50_extract.jsonl"

import json

source2chunks = {}

with open(input_file, "r", encoding = "utf-8") as file:
    for line in file:
        data = json.loads(line)
        source, id = parse_id(data["id"])

        # if source != "大染坊":
        #     # for debug 我们先取一本小说
        #     continue

        data["source"] = source
        data["seg_id"] = id

        if source not in source2chunks:
            source2chunks[source] = []
        source2chunks[source].append(data)

print(len(source2chunks["大染坊"]))

564


In [ ]:
for source, chunks in source2chunks.items():
    chunks.sort(key = lambda x: x["seg_id"])

In [ ]:
keys = list(source2chunks.keys())

# for i, source in enumerate( keys[:100]):
#     print(i, source, len(source2chunks[source]))

for source in keys:
    if len(source2chunks[source]) < 5:
        del source2chunks[source]

print(len(source2chunks))

51


In [ ]:
keys = list(source2chunks.keys())
for i, source in enumerate( keys):
    print(i, source, len(source2chunks[source]))

In [ ]:
for i in range(0, 500, 50):
    print( source2chunks["大染坊"][i]["seg_id"], end = " " )

0 50 100 150 200 250 300 350 400 450 

In [ ]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/LC1332/Zero-Haruhi
%cd /content/Zero-Haruhi

/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 435, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 435 (delta 18), reused 23 (delta 13), pack-reused 402
Receiving objects: 100% (435/435), 2.92 MiB | 19.43 MiB/s, done.
Resolving deltas: 100% (290/290), done.
/content/Zero-Haruhi


In [ ]:
print( source2chunks["大染坊"][i].keys() )

dict_keys(['text', 'id', 'model_response', 'source', 'seg_id'])


进行拼接

In [ ]:
from ChatHaruhi.novel_extract import extract_speaker, replace_recalled_dialogue

source2book = {}

from tqdm import tqdm

for source in source2chunks.keys():
    chunks = source2chunks[source]

    new_book = ""

    for chunk in tqdm(chunks):
        text = chunk["text"]
        response = chunk["model_response"]
        response = replace_recalled_dialogue(text, response)

        new_book = new_book + response + "\n"

    source2book[source] = new_book.strip()


100%|██████████| 7796/7796 [00:25<00:00, 310.22it/s]


说话人提取

In [ ]:
avoid_speakers = ['无',"“我”","未知","主人公","我","你","作家","众人","他","我们"]
narrators = ["叙述者", "旁白","文章作者","作者"]

In [ ]:
source2dialogue_list = {}

source2speakers = {}

from tqdm import tqdm

for source in tqdm(source2book.keys()):
    book = source2book[source]

    new_list = []

    lines = book.split("\n")

    last_speaker = "NOT_EXISTED_SPEAKER"

    counter = {}

    for line in lines:
        if line.strip() == "":
            continue
        speaker, dialogue = extract_speaker(line)

        if speaker is not None and speaker == last_speaker:
            last_dialogue = new_list[-1][1]
            new_dialogue = last_dialogue + "\n" + dialogue
            new_list[-1] = (speaker, new_dialogue.strip() )
        else:
            new_list.append( (speaker, dialogue) )

        # TODO 这里做个额外的屏蔽统计
        if speaker is not None and speaker not in avoid_speakers:
            if speaker not in counter:
                counter[speaker] = 0
            counter[speaker] += 1

        last_speaker = speaker

    # 根据出现次数对speaker进行排序并提取前15位
    top10_speakers = sorted(counter.items(), key=lambda item: item[1], reverse=True)[:10]


    source2dialogue_list[ source ] = new_list
    source2speakers[ source ] = top10_speakers



100%|██████████| 51/51 [00:05<00:00,  9.50it/s]


In [ ]:
keys = list(source2speakers.keys())
import random
random.shuffle( keys )
for key in keys[:10]:
    xs = [x[0] for x in source2speakers[key]]
    print(key, xs)

冰与火之歌05魔龙的狂舞 ['提利昂', '琼恩', '丹妮', '叙述者', '巴利斯坦爵士', '席恩', '阿莎', '昆廷', '戴佛斯', '布兰']
狼图腾 ['陈阵', '杨克', '叙述者', '老人', '张继原', '小狼', '包顺贵', '巴图', '乌力吉', '毕利格老人']
凤囚凰 ['楚玉', '容止', '桓远', '天如镜', '王意之', '叙述者', '花错', '观沧海', '刘子业', '越捷飞']
明朝那些事儿 ['叙述者', '朱棣', '朱元璋', '王守仁', '旁白', '作者', '徐阶', '戚继光', '胡宗宪', '崇祯']
娘要嫁人 ['齐之芳', '肖虎', '戴世亮', '李茂才', '王方', '齐之君', '齐母', '王红', '王东', '孙燕']
超级兵王 ['郭璞', '叙述者', '李长安', '宋家明', '卓不凡', '一号首长', '旁白', '米兰', '千叶子', '王思睿']
全职高手 ['叶修', '叙述者', '陈果', '旁白', '魏琛', '唐柔', '黄少天', '蓝河', '方锐', '潘林']
斗罗大陆Ⅱ绝世唐门 ['霍雨浩', '叙述者', '贝贝', '旁白', '橘子', '玄老', '王冬', '徐三石', '王冬儿', '王秋儿']
金粉世家 ['燕西', '清秋', '金太太', '凤举', '玉芬', '佩芳', '梅丽', '鹏振', '叙述者', '秀珠']
庆余年 ['范闲', '叙述者', '皇帝', '陈萍萍', '海棠', '林婉儿', '言冰云', '范若若', '五竹', '范建']


In [ ]:
from transformers import AutoTokenizer

model_name = 'silk-road/Haruhi-Zero-7B-0_3'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/silk-road/Haruhi-Zero-7B-0_3:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/64.0 [00:00<?, ?B/s]

In [ ]:
def calculate_tokenized_length(text):
    if text == None:
        return 0
    tokens = tokenizer.encode(text)
    return len(tokens)

In [ ]:
MAX_TOKEN_PER_MEMEORY = 500

source2memory_lists = {}

import copy

for source in tqdm(source2book.keys()):
    speakers = source2speakers[source]
    speakers = [x[0] for x in speakers]
    dialogue_list = source2dialogue_list[ source ]

    dialogue_token_len = [ calculate_tokenized_length(x[1]) for x in dialogue_list ]

    speaker2memory_lists = {}


    for target_speaker in speakers:
        lids = []

        for i, dialogue in enumerate( dialogue_list ):
            speaker = dialogue[0]
            if speaker is not None and target_speaker in speaker:
                lids.append(i)

        minimal_token_consider = [] # 每个lids和lids前面一句话的token之和
        maximal_token_consider = [] # 每个lids到前一个lids的token之和

        reverse_lids = lids[::-1]

        for i in range(len(reverse_lids)):
            current_lid = reverse_lids[i]
            last_lids = reverse_lids[i+1] if i+1 < len(reverse_lids) else 0

            last_lids_next_line = last_lids + 1

            current_token_len = dialogue_token_len[current_lid]
            last_token_len = dialogue_token_len[current_lid-1] if current_lid > 0 else 0

            minimal_token_consider.append( current_token_len + last_token_len )
            maximal_token_consider.append( sum( dialogue_token_len[last_lids_next_line:current_lid+1] ) )

        saved_memory_lists = []

        current_memory_token_len = 0
        current_memory_list = []
        for i in range(len(reverse_lids)):
            maximal_token = maximal_token_consider[i]
            minimal_token = minimal_token_consider[i]

            current_lid = reverse_lids[i]
            last_line = current_lid - 1 if current_lid > 0 else None
            last_lid = reverse_lids[i+1] if i+1 < len(reverse_lids) else 0

            if maximal_token + current_memory_token_len <= MAX_TOKEN_PER_MEMEORY:
                for j in range(reverse_lids[i], last_lid,-1):
                    current_memory_list.append(j)
                    current_memory_token_len += dialogue_token_len[j]
                continue

            if len(current_memory_list) > 0 and  current_memory_token_len + minimal_token > MAX_TOKEN_PER_MEMEORY:
                current_memory_list.reverse()
                save_memory = copy.deepcopy(current_memory_list)
                saved_memory_lists.append( save_memory )
                current_memory_list = []
                current_memory_token_len = 0

            if maximal_token > MAX_TOKEN_PER_MEMEORY:
                # 直接切行
                special_token_count = 0

                save_memory = []
                if last_line:
                    special_token_count += dialogue_token_len[last_line]
                    save_memory.append( last_line )

                new_text = ""
                speaker = dialogue_list[current_lid][0]
                lines = dialogue_list[current_lid][1].split("\n")
                for line in lines:
                    special_token_count += calculate_tokenized_length(line)
                    if special_token_count < MAX_TOKEN_PER_MEMEORY:
                        new_text += line + "\n"
                    else:
                        break

                dialogue_list[current_lid] = (speaker, new_text.strip())
                save_memory.append(current_lid)

                saved_memory_lists.append( save_memory )
                continue

            # 否则则说明 current_memory_token_len + minimal_token < MAX_TOKEN_PER_MEMEORY < maximal_token + current_memory_token_len

            for j in range(reverse_lids[i], last_lid,-1):
                current_memory_token_len += dialogue_token_len[j]
                if current_memory_token_len > MAX_TOKEN_PER_MEMEORY:
                    break
                else:
                    current_memory_list.append(j)

            current_memory_list.reverse()
            save_memory = copy.deepcopy(current_memory_list)
            saved_memory_lists.append( save_memory )
            current_memory_list = []
            current_memory_token_len = 0

        if len(current_memory_list) > 0:
            current_memory_list.reverse()
            save_memory = copy.deepcopy(current_memory_list)
            saved_memory_lists.append( save_memory )

        memory_lists = []

        for memory_list_id in saved_memory_lists:
            memory_list = []
            for lid in memory_list_id:
                memory_list.append(dialogue_list[lid])
            memory_lists.append(memory_list)



        speaker2memory_lists[target_speaker] = memory_lists

    source2memory_lists[source] = speaker2memory_lists

100%|██████████| 51/51 [02:26<00:00,  2.88s/it]


In [ ]:
print(memory_list_id)


[73, 74, 75, 76, 77]


In [ ]:
def memory_list2text( memory_list ):
    line_set = set()
    new_text = ""
    for mem in memory_list:
        spk = mem[0]
        text = mem[1]
        if spk is None:
            newline = text + "\n"
            # new_text += text + "\n"
        else:
            # new_text += f"{spk}: {text}\n"
            newline = f"{spk}: {text}\n"

        newline = newline.replace("*说*","")
        if newline not in line_set:
            new_text += newline
            line_set.add(newline)
    return new_text.strip()

def memory_list2conversations( memory_list , target_speaker ):

    line_set = set()

    conversations = []

    current_value = ""

    for mem in memory_list:
        spk = mem[0]
        text = mem[1]

        if text in line_set:
            continue
        line_set.add(text)

        if spk is not None and target_speaker == spk.strip() and calculate_tokenized_length(text) >= 8:
            conversations.append({"from" : "human" , "value":current_value.strip()})
            conversations.append({"from" : "gpt" , "value":text.strip()})
            current_value = ""
            continue

        if spk is None:
            current_value += text + "\n"
        else:
            current_value += f"{spk}: {text}\n"

    for conv in conversations:
        conv["value"] = conv["value"].replace("*说*","")

    while len(conversations) > 0 and conversations[-1]["from"] == "human":
        del conversations[-1]
        if len(conversations) == 0:
            return []

    return conversations

In [ ]:
for speaker in speaker2memory_lists:
    print(speaker, " ", len(speaker2memory_lists[speaker]))

顾慎为   3910
上官如   845
上官飞   728
木老头   656
叙述者   599
荷女   618
龙王   728
韩芬   346
方闻是   308
罗宁茶   234


In [ ]:
from ChatHaruhi.embeddings import get_general_embeddings_safe

In [ ]:
def package_persona( role_name, world_name ):
    return f"""I want you to act like {role_name} from {world_name}.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like {role_name} using the tone, manner and vocabulary {role_name} would use."""

def package_persona_for_narrator( role_name, world_name ):
    return f"""I want you to act like narrator {role_name} from {world_name}.
当角色行动之后，继续交代和推进新的剧情."""

def package_system_prompt( persona, role_name ):
    return f"""You are now in roleplay conversation mode. Pretend to be {role_name} whose persona follows:
{persona}

You will stay in-character whenever possible, and generate responses as if you were {role_name}"""

In [ ]:
from typing import List
import numpy as np

def normalize_vecs(vecs: List[List[float]]) -> List[List[float]]:
    """Normalize a list of vectors."""
    normalized_vecs = []
    for vec in vecs:
        norm = np.linalg.norm(vec)
        if norm > 0:
            normalized_vecs.append((np.array(vec) / norm).tolist())
        else:
            normalized_vecs.append(vec)  # Handle zero vectors if any
    return normalized_vecs


# Since vecs are normalized, we can simplify the cosine similarity function for use in nn_search
def nn_search_normalized(query_vec: List[float], avoid_ids: List[int], vecs: List[List[float]]) -> int:
    """Find the index of the vector in normalized vecs that is most similar to query_vec, excluding the vector at avoid_id."""
    # query_vec_normalized = np.array(query_vec) / np.linalg.norm(query_vec)  # Normalize query_vec
    similarities = [np.dot(query_vec, vec) if idx not in avoid_ids else -np.inf for idx, vec in enumerate(vecs)]
    return np.argmax(similarities)


In [ ]:
import os
import json
import random
def sample_and_save( save_datas, save_name ,sample_n=50, save_folder = "/content/drive/MyDrive/CardBuild/HaruhiZero"):
    file_name = os.path.join( save_folder, save_name + ".jsonl" )
    with open( file_name, "w", encoding="utf-8" ) as f:
        for data in save_datas:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

    sample_name = os.path.join( "/content/", save_name + "_sample.jsonl" )
    with open( sample_name, "w", encoding="utf-8" ) as f:
        random_50 = random. sample( save_datas, sample_n )
        for data in random_50:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

In [ ]:
from IPython.display import clear_output

from tqdm import tqdm

import random

dsp_count = 0

save_datas = []

current_part_id = 0

save_name = "novel_50_to_conversation"
book_count = 0

for source in source2book.keys():
    speaker2memory_lists = source2memory_lists[source]

    clear_output()

    print("处理 ", source, "第", book_count, "本")
    book_count += 1

    print("当前数据量 ", len(save_datas))

    for speaker in speaker2memory_lists:
        memory_lists = speaker2memory_lists[speaker]

        rag_texts = [memory_list2text(mem) for mem in memory_lists]
        convs = [memory_list2conversations(mem, speaker) for mem in memory_lists]

        if speaker in narrators:
            persona_head = package_persona_for_narrator( speaker, source ) + "\nClassic scenes for the role are as follows:\n"
        else:
            persona_head = package_persona( speaker, source ) + "\nClassic scenes for the role are as follows:\n"

        model_name = "BAAI/bge-small-zh-v1.5"

        if len(rag_texts) < 600:
            vecs = get_general_embeddings_safe( rag_texts, model_name = model_name )

            # Normalize the example vecs
            normalized_vecs = normalize_vecs(vecs)
        else:
            vecs = [[] for _ in range(len(rag_texts))]
            normalized_vecs = [[] for _ in range(len(rag_texts))]

        personas = [ persona_head ] * len(rag_texts)

        n = len(rag_texts)

        for rag_index in range(2):
            deal_index = [i for i in range(n)]
            random.shuffle(deal_index)

            current_used_ids = []

            if len(deal_index) < 600:
                for i in tqdm(deal_index):
                    if len(convs[i]) == 0:
                        continue

                    top1 = nn_search_normalized( normalized_vecs[i] , current_used_ids + [i] , normalized_vecs )
                    current_used_ids.append(top1)

                    personas[i] += f"###\n{rag_texts[top1]}\n"
            else:
                match_index = [i for i in range(n)]
                random.shuffle(match_index)
                for i in tqdm(deal_index):
                    if len(convs[i]) == 0:
                        continue

                    top1 = match_index[i]
                    current_used_ids.append(top1)

                    personas[i] += f"###\n{rag_texts[top1]}\n"


        source_str = f"{source}_{speaker}"
        for i in range(n):

            if len(convs[i]) == 0:
                continue

            system_prompt = package_system_prompt( personas[i], speaker )

            save_data = {
                "source":source_str,
                "conversations":convs[i],
                "system":system_prompt
            }

            save_datas.append(save_data)

    if len(save_datas) >= 100000:
        save_name = "novel_50_to_conversation_part" + str(current_part_id)
        current_part_id += 1
        sample_and_save( save_datas, save_name , sample_n = 50)
        del save_datas
        save_datas = []



处理  超级兵王 第 50 本
当前数据量  31534


100%|██████████| 239/239 [00:03<00:00, 73.24it/s] 


In [ ]:
if len(save_datas) >= 0:
    save_name = "novel_50_to_conversation_part" + str(current_part_id)
    current_part_id += 1
    sample_and_save( save_datas, save_name , sample_n = 50)
    del save_datas
    save_datas = []

In [ ]:


sample_and_save( save_datas, "novel_50_to_conversation" , sample_n = 50)

In [ ]:
for data in save_datas:
    if len(data["conversations"]) % 2 != 0:
        print(data)
        break
    elif data["conversations"][0]["from"] == "gpt":
        print(data)
        break
    elif data["conversations"][-1]["from"] == "human":
        print(data)
        break

In [ ]:
for data in save_datas[]:
    print(data["source"])
    print(data["conversations"])
    print(data["system"])
    print()

100%|██████████| 33/33 [00:03<00:00,  9.92it/s]


In [ ]:
avoid_ids = []

n = len(vecs)
import random



print(deal_index[:4])

rag_ids = []

current_used_ids = []

from tqdm import tqdm
for i in tqdm(deal_index):
    top1 = nn_search_normalized( normalized_vecs[i] , current_used_ids + [i] , normalized_vecs )
    current_used_ids.append(top1)
    rag_ids.append(top1)

[258, 237, 189, 85]


100%|██████████| 514/514 [00:19<00:00, 26.59it/s] 


In [ ]:
print(rag_ids[:10])

[257, 438, 311, 89, 413, 452, 127, 30, 308, 221]


vecs是一个list of list of float， 每个元素是一个embedding向量

我希望实现一个 nn_search( query_vec , avoid_id )

其中query_vec是一个list of float

返回除了avoid_id对应的元素之外，和query_vec在cosine距离上最接近的元素

In [ ]:
print(vecs[0])

[0.00857368391007185, 0.024749422445893288, 0.0389234833419323, 0.01729436218738556, -0.04563982039690018, -0.027840416878461838, -0.02191009745001793, 0.0038269469514489174, -0.015359637327492237, 0.03595023602247238, 0.09147959202528, -0.3811834752559662, 0.009313845075666904, -0.018114736303687096, 0.013250540010631084, -0.04310958832502365, 0.0006790618062950671, 0.00469238730147481, 0.03925265744328499, 0.01733214594423771, 0.0264841690659523, -0.031113404780626297, -0.04992251098155975, 0.03794755041599274, 0.029948368668556213, -0.06283234804868698, 0.036146096885204315, -0.03006013296544552, -0.06511688232421875, -0.019239582121372223, 0.008901293389499187, -0.0629120022058487, -0.023375675082206726, -0.040794581174850464, -0.012890283018350601, -0.03488422930240631, -0.03141654655337334, -0.04510388523340225, 0.019871355965733528, -0.019697867333889008, -0.04139396920800209, -0.007770806085318327, -0.04370585456490517, 0.010295907966792583, -0.04054240509867668, 0.030483869835

In [ ]:
print(len(rag_texts))
print(len(convs))

514
514


In [ ]:
print(len(memory_lists))

514


In [ ]:
a = [1,2,3,4]
del a[-1]
print(a)

[1, 2, 3]


In [ ]:
print(lids)

print(reverse_lids)

print(saved_memory_lists)

print(len(saved_memory_lists))

for mem_list, memory_list in zip(saved_memory_lists[:10],memory_lists[:10]):
    print()
    print(mem_list)
    count = 0
    for lid in mem_list:
        l = calculate_tokenized_length(  dialogue_list[lid][1] )
        count += l
        print(f"{l}+" , end = "")

    print("=", count)

    # print(memory_list2text(memory_list))
    conversations = memory_list2conversations(memory_list, target_speaker)
    for conv in conversations:
        print(conv["from"], "\t", conv["value"])

[3821, 3824, 3826, 3828, 3830, 3832, 3834, 3836, 3839, 3841, 3917, 3921, 3929, 3940, 3965, 4045, 4047, 4049, 4051, 4055, 4057, 4059, 4061, 4063, 4065, 4067, 4069, 4071, 4073, 4322, 4333, 4335, 4337, 4810, 4812, 4814, 4816, 4818, 5562, 5564, 5566, 5568, 5570, 5572, 5574, 5729, 5731, 5733, 5735, 5737, 5739, 5741, 6077, 6079, 6081, 6083, 6085, 6087, 6089, 6091, 6136, 6138, 6220, 6222, 6291, 6295, 6297, 6300, 6302, 6308, 6310, 6312, 6314, 6316, 6378, 6380, 6382, 6384, 6386, 6388, 6390, 6438, 6440, 6442, 6444, 6446, 6448, 6450, 6452, 6454, 6456, 6458, 6482, 6484, 6486, 6488, 6490, 6492, 6494, 6500, 6535, 6537, 6539, 6541, 6542, 6544, 6546, 6548, 6550, 6805, 6807, 6809, 6811, 6813, 6815, 6819, 6821, 6888, 6890, 6991, 6993, 6999, 7006, 7053, 7072, 7078, 7080, 7082, 7084, 7086, 7088, 7092, 7094, 7096, 7109, 7111, 7113, 7114, 7117, 7119, 7123, 7126, 7128, 7170, 7172, 7174]
[7174, 7172, 7170, 7128, 7126, 7123, 7119, 7117, 7114, 7113, 7111, 7109, 7096, 7094, 7092, 7088, 7086, 7084, 7082, 7080, 70

In [ ]:
a = [1,2,3,4]
a.reverse()
print(a)

In [ ]:
print(data)
print(data.keys())
print(data["id"])
print(data["id"].rfind("_"))